In [302]:
import pandas as pd
import numpy as np
import seaborn as sns
datapath = "./data/"

In [316]:
df = pd.read_csv(datapath + "Fuel_Consumption_2000-2022.csv") # https://www.kaggle.com/datasets/ahmettyilmazz/fuel-consumption
df_prices = pd.read_csv(datapath + "Fuel_prices_Canada.csv") # https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=1810000101

In [312]:

def subset_price_df(df_prices):
    # Only keep the date, type of fuel and cost
    df_prices = df_prices[["REF_DATE","Type of fuel", "VALUE"]]

    # Only keep values in the interval 2000-2022
    df_prices['REF_DATE'] = pd.to_datetime(df_prices['REF_DATE'], format='%Y-%m')
    # df_prices.loc[:, 'REF_DATE'] = pd.to_datetime(df_prices['REF_DATE'], format='%Y-%m')

    df_prices = df_prices.loc[(df_prices['REF_DATE'] >= '2000-01')
                        & (df_prices['REF_DATE'] < '2023-01')]

    # Only keep the values from self-service stations
    df_prices = df_prices[df_prices['Type of fuel'].str.contains('self service')]
    return df_prices

def reformat_price_df(df):
    df['REF_DATE'] = df['REF_DATE'].dt.year
    replacement_dict = {
        'Diesel fuel at self service filling stations': 'D',
        'Premium unleaded gasoline at self service filling stations': 'Z',
        'Regular unleaded gasoline at self service filling stations': 'X'
    }
    df['Type of fuel'] = df['Type of fuel'].map(replacement_dict)

    df = df.rename(columns={"REF_DATE":"YEAR", "Type of fuel":"FUEL", "VALUE":"FUEL COST"})
    return df

def aggregate_price_df(df):
    return df.groupby(['FUEL','YEAR']).mean()


Preprocessing

In [313]:
# Drop records with NAs
def clean_dataset(df: pd.DataFrame) -> pd.DataFrame:
    return df.dropna()

# Drop unused columns
def remove_features(df: pd.DataFrame) -> pd.DataFrame:
    return df.drop(["CYLINDERS", "MODEL", "CYLINDERS", "TRANSMISSION", "HWY (L/100 km)", "COMB (L/100 km)", "COMB (mpg)"], axis=1)

# Remove subsections of classes
def remove_class_subsections(df: pd.DataFrame) -> pd.DataFrame:
    df['VEHICLE CLASS'] = df['VEHICLE CLASS'].str.split(': ').str[0]
    df['VEHICLE CLASS'] = df['VEHICLE CLASS'].str.split(' - ').str[0]
    df['VEHICLE CLASS'] = df['VEHICLE CLASS'].str.upper()
    return df

# Subset price dataset to match vehicle data
def subset_price_df(df_prices:pd.DataFrame) -> pd.DataFrame:
    # Only keep the date, type of fuel and cost
    df_prices = df_prices[["REF_DATE","Type of fuel", "VALUE"]]

    # Only keep values in the interval 2000-2022
    df_prices['REF_DATE'] = pd.to_datetime(df_prices['REF_DATE'], format='%Y-%m')
    # df_prices.loc[:, 'REF_DATE'] = pd.to_datetime(df_prices['REF_DATE'], format='%Y-%m')

    df_prices = df_prices.loc[(df_prices['REF_DATE'] >= '2000-01')
                        & (df_prices['REF_DATE'] < '2023-01')]

    # Only keep the values from self-service stations
    df_prices = df_prices[df_prices['Type of fuel'].str.contains('self service')]
    return df_prices

# Reformat price dataframe to match vehicle dataframe
def reformat_price_df(df:pd.DataFrame) -> pd.DataFrame:
    df['REF_DATE'] = df['REF_DATE'].dt.year
    replacement_dict = {
        'Diesel fuel at self service filling stations': 'D',
        'Premium unleaded gasoline at self service filling stations': 'Z',
        'Regular unleaded gasoline at self service filling stations': 'X'
    }
    df['Type of fuel'] = df['Type of fuel'].map(replacement_dict)

    df = df.rename(columns={"REF_DATE":"YEAR", "Type of fuel":"FUEL", "VALUE":"FUEL COST"})
    return df

# Aggregate price dataframe based on fuel type and year
def aggregate_price_df(df: pd.DataFrame) -> pd.DataFrame:
    return df.groupby(['FUEL','YEAR']).mean()



In [314]:
def preprocess_data(df: pd.DataFrame):
    return(
        df
        .pipe(clean_dataset)
        .pipe(remove_features)
        .pipe(remove_class_subsections)
    )

def preprocess_price_data(df: pd.DataFrame):
    return(
        df
        .pipe(subset_price_df)
        .pipe(reformat_price_df)
        .pipe(aggregate_price_df)
    )

df = preprocess_data(df)
df_prices = preprocess_price_data(df_prices)

df_merged = df.merge(df_prices, on=['FUEL', 'YEAR'], how='left')


C:\Users\anton\AppData\Local\Temp\ipykernel_13120\1600240224.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prices['REF_DATE'] = pd.to_datetime(df_prices['REF_DATE'], format='%Y-%m')


In [315]:
df_merged

,YEAR,MAKE,VEHICLE CLASS,ENGINE SIZE,FUEL,FUEL CONSUMPTION,EMISSIONS,FUEL COST
0,2000,ACURA,COMPACT,1.6,X,9.2,186,72.834722
1,2000,ACURA,COMPACT,1.6,X,8.5,175,72.834722
2,2000,ACURA,MID-SIZE,3.2,Z,12.2,230,81.142130
3,2000,ACURA,MID-SIZE,3.5,Z,13.4,264,81.142130
4,2000,ACURA,SUBCOMPACT,1.8,X,10.0,198,72.834722
...,...,...,...,...,...,...,...,...
22551,2022,Volvo,SUV,2.0,Z,10.7,219,192.639815
22552,2022,Volvo,SUV,2.0,Z,10.5,219,192.639815
22553,2022,Volvo,SUV,2.0,Z,11.0,232,192.639815
22554,2022,Volvo,SUV,2.0,Z,11.5,236,192.639815
